# Урок 13. Модель BERT и GPT
## Домашнее задание.

1. Взять датасет https://huggingface.co/datasets/merionum/ru_paraphraser решить задачу парафраза  
2. (дополнительно необязательная задача)на выбор:  
    - взять https://huggingface.co/datasets/sberquad https://huggingface.co/datasets/blinoff/medical_qa_ru_data натренировать любую модель для вопросно-ответной системы
    - как альтернатива можно взять любой NER датасет из https://github.com/natasha/corus#reference и обучить NER

### Задание 1.

In [1]:
!pip install transformers sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 87.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 99.1 MB/s eta 0:00:00


In [2]:
from transformers import MBartTokenizer, MBartForConditionalGeneration

model_name = "IlyaGusev/mbart_ru_sum_gazeta"
tokenizer = MBartTokenizer.from_pretrained(model_name)

In [5]:
model = MBartForConditionalGeneration.from_pretrained(model_name)
model.to("cuda")

MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): Embedding(250027, 1024, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): Embedding(250027, 1024, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0): MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerN

In [6]:
article_text = 'На данный момент Китай очень сильно нуждается в импульсе для роста экономики, поэтому Пекин намеревается наладить отношения с Европой и дистанцироваться от Москвы. Об этом сообщает издание The New York Times (NYT). Отмечается, что в ходе своего недавнего турне по странам Европы глава МИД Китая Ван И пытался «привлечь европейские страны для возрождения своей экономики», а также нахождения «точек соприкосновения». Эксперты агентства напомнили, что Ван И отказался напрямую поддерживать Россию. Тогда китайский дипломат заявлял, что Пекин не хочет, чтобы кризис затянулся и обострился. При этом NT сообщает, что Соединенные штаты оказывают серьезное давление на Китай, несмотря на попытки наладить диалог между странами. Спустя почти двух лет «прохладных отношений» КНР и Европы и украинского конфликта аналитики надеялись, что Пекин смягчит свой тон и покажет, что начинает дистанцироваться от Москвы.'
input_ids = tokenizer(
    [article_text],
    max_length=600,
    truncation=True,
    return_tensors="pt",
)["input_ids"].to("cuda")
print(input_ids)

tensor([[   672,  67558,   8270,  57592,   4296,  38519,  35240,   4354,     49,
             35,  47926,  29524,   2687,    518, 106525,  81491,      4,  22978,
         206440,  70438,  10653,   4354,     29,  13702,   3410,  27546,    135,
              6, 132272,   1768,     35, 145894, 133529,    183,  80046,      5,
           5188,   3920, 106064,  61040,    581,   2356,   5753,  19114,     15,
          25182,    618,    194,   3858,  56894,   4354,      4,    414,     49,
          68642,  24840, 202123,  57803,   8526,    336,    129,   8005,    130,
          96565,  41976,   1435,  56934, 220115,  85902,    804, 206669,     94,
           7171, 137753,  61683,    103,  30296,    518,  11125, 121522,   1114,
          14879,  81491,   1339,    252,   5058,     29,  33394,  38847,     94,
            328,  33963,    529,   7171,    698, 125802,  13617,   1251,      6,
         182678,    227, 158193,     29,  41883,    546,      4,    414,  85902,
            804,  67589,   6

In [7]:
output_ids = model.generate(
    input_ids=input_ids,
    no_repeat_ngram_size=4
)[0]
summary = tokenizer.decode(output_ids, skip_special_tokens=True)
print(summary)

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 200 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Китай планирует дистанцироваться от Москвы и наладить диалог с Европой. Об этом сообщает The New York Times (NYT). Ранее глава МИД Китая Ван И отказался напрямую поддерживать Россию, поскольку Пекин не хочет, чтобы кризис затянулся и обострился.
